# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [28]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])

from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score


from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /users/yanlingl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /users/yanlingl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('df', engine,index_col=0)

In [30]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
X = df['message']
Y = df[df.columns[4:-1]]

### 2. Write a tokenization function to process your text data

In [32]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                     ('tfidf',TfidfTransformer()),
                     ('MOC', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [35]:
print(X_train.shape)
print(y_train.shape)

(19662,)
(19662, 35)


In [36]:
%%time
pipeline.fit(X_train, y_train)

CPU times: user 7min 36s, sys: 560 ms, total: 7min 37s
Wall time: 7min 37s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x2aaaf62079e0>)),
                ('tfidf', TfidfTransformer()),
                ('MOC',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
predicted=pipeline.predict(X_test)

In [11]:
overall_accuracy = (predicted == y_test).mean().mean()
overall_accuracy

0.9484284406469332

In [12]:
print(classification_report(y_test.values, predicted, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.80      0.97      0.88      4984
               request       0.91      0.44      0.59      1134
                 offer       0.00      0.00      0.00        29
           aid_related       0.79      0.62      0.70      2691
          medical_help       0.72      0.06      0.10       522
      medical_products       0.63      0.06      0.11       317
     search_and_rescue       0.60      0.03      0.06       177
              security       0.50      0.01      0.02       114
              military       0.67      0.06      0.12       190
           child_alone       0.00      0.00      0.00         0
                 water       0.94      0.23      0.37       429
                  food       0.90      0.42      0.57       729
               shelter       0.88      0.26      0.40       581
              clothing       0.71      0.06      0.11        87
                 money       0.80      

/users/yanlingl/.conda/envs/yl_ML/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/yanlingl/.conda/envs/yl_ML/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/yanlingl/.conda/envs/yl_ML/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users

In [13]:
#see if the trained model produces reasonable result.
msg = ['Help! I was trapped in my apartment there is fire ']
test_output = pipeline.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related']


In [14]:
msg = [' Hello I see fire in the street and many houses are destroyed, homeless people everywhere']
test_output = pipeline.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'aid_related']


In [15]:
result={}
for i in range(predicted.shape[1]):
    result['cat_'+str(i)]=classification_report(y_test[y_test.columns[i]], predicted[:,i])
result

{'cat_0': '              precision    recall  f1-score   support\n\n           0       0.73      0.25      0.37      1570\n           1       0.80      0.97      0.88      4984\n\n    accuracy                           0.80      6554\n   macro avg       0.77      0.61      0.63      6554\nweighted avg       0.79      0.80      0.76      6554\n',
 'cat_1': '              precision    recall  f1-score   support\n\n           0       0.89      0.99      0.94      5420\n           1       0.91      0.44      0.59      1134\n\n    accuracy                           0.89      6554\n   macro avg       0.90      0.71      0.76      6554\nweighted avg       0.90      0.89      0.88      6554\n',
 'cat_2': '              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00      6525\n           1       0.00      0.00      0.00        29\n\n    accuracy                           1.00      6554\n   macro avg       0.50      0.50      0.50      6554\nweighted avg  

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x2aaaee6bcf80>)),
  ('tfidf', TfidfTransformer()),
  ('MOC', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x2aaaee6bcf80>),
 'tfidf': TfidfTransformer(),
 'MOC': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [17]:
%%time
parameters = {
    'MOC__estimator__n_estimators': [50,100],
    'MOC__estimator__criterion':['gini','entropy'] ,#MOC__criterion
    'vect__max_features': [None,'auto', 5000]
}

cv =  GridSearchCV(pipeline, parameters,  cv=3, verbose=2, scoring='f1_micro') #scoring='f1_micro',, n_jobs=-1
cv.fit(X_train, y_train)


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] MOC__estimator__criterion=gini, MOC__estimator__n_estimators=50, vect__max_features=None 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  MOC__estimator__criterion=gini, MOC__estimator__n_estimators=50, vect__max_features=None, total= 2.3min
[CV] MOC__estimator__criterion=gini, MOC__estimator__n_estimators=50, vect__max_features=None 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.3min remaining:    0.0s


[CV]  MOC__estimator__criterion=gini, MOC__estimator__n_estimators=50, vect__max_features=None, total= 2.3min
[CV] MOC__estimator__criterion=gini, MOC__estimator__n_estimators=50, vect__max_features=None 
[CV]  MOC__estimator__criterion=gini, MOC__estimator__n_estimators=50, vect__max_features=None, total= 2.3min
[CV] MOC__estimator__criterion=gini, MOC__estimator__n_estimators=50, vect__max_features=auto 


...

/users/yanlingl/.conda/envs/yl_ML/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/users/yanlingl/.conda/envs/yl_ML/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/users/yanlingl/.conda/envs/yl_ML/lib/python3.7/site-packages/sklearn/pipeline.py", line 330, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/users/yanlingl/.conda/envs/yl_ML/lib/python3.7/site-packages/sklearn/pipeline.py", line 296, in _fit
    **fit_params_steps[name])
  File "/users/yanlingl/.conda/envs/yl_ML/lib/python3.7/site-packages/joblib/memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "/users/yanlingl/.conda/envs/yl_ML/lib/python3.7/site-packages/sklearn/pipeline.py", line 

[CV]  MOC__estimator__criterion=entropy, MOC__estimator__n_estimators=100, vect__max_features=auto, total=   5.0s
[CV] MOC__estimator__criterion=entropy, MOC__estimator__n_estimators=100, vect__max_features=auto 
[CV]  MOC__estimator__criterion=entropy, MOC__estimator__n_estimators=100, vect__max_features=auto, total=   5.0s
[CV] MOC__estimator__criterion=entropy, MOC__estimator__n_estimators=100, vect__max_features=auto 
[CV]  MOC__estimator__criterion=entropy, MOC__estimator__n_estimators=100, vect__max_features=auto, total=   5.1s
[CV] MOC__estimator__criterion=entropy, MOC__estimator__n_estimators=100, vect__max_features=5000 
[CV]  MOC__estimator__criterion=entropy, MOC__estimator__n_estimators=100, vect__max_features=5000, total= 3.8min
[CV] MOC__estimator__criterion=entropy, MOC__estimator__n_estimators=100, vect__max_features=5000 
[CV]  MOC__estimator__criterion=entropy, MOC__estimator__n_estimators=100, vect__max_features=5000, total= 3.8min
[CV] MOC__estimator__criterion=ent

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 75.2min finished


CPU times: user 1h 21min 1s, sys: 5.57 s, total: 1h 21min 6s
Wall time: 1h 21min 6s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x2aaaee6bcf80>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('MOC',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'MOC__estimator__criterion': ['gini', 'entropy'],
                         'MOC__estimator__n_estimators': [50, 100],
                         'vect__max_features': [None, 'auto', 5000]},
             scoring='f1_micro', verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
y_pred = cv.predict(X_test)
overall_accuracy = (y_pred == y_test).mean().mean()
overall_accuracy

0.9513448711800864

In [19]:
print(classification_report(y_test.values, y_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      4984
               request       0.90      0.46      0.61      1134
                 offer       0.00      0.00      0.00        29
           aid_related       0.76      0.65      0.70      2691
          medical_help       0.74      0.10      0.18       522
      medical_products       0.78      0.16      0.27       317
     search_and_rescue       0.66      0.12      0.20       177
              security       0.33      0.01      0.02       114
              military       0.63      0.14      0.23       190
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.40      0.55       429
                  food       0.87      0.59      0.70       729
               shelter       0.84      0.45      0.59       581
              clothing       0.75      0.10      0.18        87
                 money       0.60      

/users/yanlingl/.conda/envs/yl_ML/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/yanlingl/.conda/envs/yl_ML/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/yanlingl/.conda/envs/yl_ML/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users

In [20]:
msg = ['Help! I was trapped in my apartment there is fire ']
test_output = cv.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related']


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
cv.best_params_

{'MOC__estimator__criterion': 'gini',
 'MOC__estimator__n_estimators': 100,
 'vect__max_features': 5000}

### 9. Export your model as a pickle file

In [22]:
import pickle
filename = 'model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [37]:
%%time
# It takes far longer than the default setting to train...
pipeline2 = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                         ('tfidf',TfidfTransformer()),
                         ('MOC', MultiOutputClassifier(RandomForestClassifier(criterion= 'gini', n_estimators= 100, max_features=5000)))])
pipeline2.fit(X_train, y_train)

CPU times: user 1h 5s, sys: 1.06 s, total: 1h 6s
Wall time: 1h 6s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x2aaaf62079e0>)),
                ('tfidf', TfidfTransformer()),
                ('MOC',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_features=5000)))])